## Task-7. Дисперсионный анализ

In [2]:
import helper as hp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import math

In [3]:
my_var = 13

In [4]:
df = hp.get_data('7', hp.get_tasks_var(my_var))
pooled = df['A8'].to_numpy()
df

,A4,A8
0,IIB,382
1,I,508
2,IIA,415
3,IIA,498
4,IIA,350
...,...,...
1068,IIB,349
1069,I,439
1070,IIB,383
1071,IIB,315


In [5]:
tabl = pd.DataFrame({"n": df.groupby('A4').size()}
            ).join(df.groupby('A4').mean().rename(columns={'A8':'mean'})
                  ).join(df.groupby('A4').var().rename(columns={'A8':'variance'}))
tabl

,n,mean,variance
A4,,,
I,180,533.666667,7652.245810
IIA,359,440.818942,4684.098411
IIB,534,383.829588,4975.631318


In [6]:
k = 3
i = df.groupby('A4').get_group('I')['A8'].to_numpy()
iia = df.groupby('A4').get_group('IIA')['A8'].to_numpy()
iib = df.groupby('A4').get_group('IIB')['A8'].to_numpy()

In [7]:
group_i = [i, iia, iib]
within_group_var = np.sum([x.size * x.var() for x in group_i]) / pooled.size
between_group_var = np.sum([x.size * (x.mean() - pooled.mean())**2 for x in group_i]) / pooled.size
pooled_var = pooled.var()

In [8]:
pd.DataFrame({
    "Источник вариации" : ["Группиров. приз", "Остаточ. факторы", "Все факторы"],
    "Мера вариации" : [between_group_var, within_group_var, pooled_var],
    "Число степ. своб." : [k - 1, pooled.size - k, pooled.size - 1],
    "Несмещ. оценка" : [pooled.size / (k - 1) * between_group_var, 
                        pooled.size / (pooled.size - k) * within_group_var,
                        pooled.size / (pooled.size - 1) * pooled_var],
})

,Источник вариации,Мера вариации,Число степ. своб.,Несмещ. оценка
0,Группиров. приз,2898.992670,2,1.555310e+06
1,Остаточ. факторы,5310.969920,1070,5.325860e+03
2,Все факторы,8209.962589,1072,8.217621e+03


In [9]:
print("True" if pooled_var == between_group_var + within_group_var else "False")
pd.DataFrame({
    "D_межгр" : [between_group_var],
    "D_внгруп" : [within_group_var],
    "D_общ" : [pooled_var],
    "D_межгр + D_внгруп" : [between_group_var + within_group_var]
})

True


,D_межгр,D_внгруп,D_общ,D_межгр + D_внгруп
0,2898.99267,5310.96992,8209.962589,8209.962589


In [10]:
pd.DataFrame({
    "Эмпирический коэффициент детерминации" : [between_group_var / pooled_var],
    "Эмпирическое корреляционное отношение" : [np.sqrt(between_group_var / pooled_var)],
})

,Эмпирический коэффициент детерминации,Эмпирическое корреляционное отношение
0,0.353107,0.594228


In [11]:
anv = stats.f_oneway(i, iia, iib)
anv

F_onewayResult(statistic=292.02972372240845, pvalue=6.244173657733121e-102)

In [12]:
pd.DataFrame({
    "Cтат. крит." : [anv.statistic], 
    "p-value" : [anv.pvalue],
    "Стат. реш. при alpha=0.01" : ["Отклоняется" if anv.pvalue < 0.01 else "Принимается"],
    "Ошибка стат. решения" : ["-"],
})

,Cтат. крит.,p-value,Стат. реш. при alpha=0.01,Ошибка стат. решения
0,292.029724,6.244174e-102,Отклоняется,-


In [20]:
pd.DataFrame({
    "Cтат. крит." : [anv.statistic]*3, 
    "p-value" : [anv.pvalue]*3,
    "Стат. реш. при" : ["Отклоняется" if anv.pvalue < 0.01 else "Принимается", 
                        "Отклоняется" if anv.pvalue < 0.05 else "Принимается",
                        "Отклоняется" if anv.pvalue < 0.1 else "Принимается"],
    "Ошибка стат. решения" : ["-", "-", "-"],
})

,Cтат. крит.,p-value,Стат. реш. при,Ошибка стат. решения
0,292.029724,6.244174e-102,Отклоняется,-
1,292.029724,6.244174e-102,Отклоняется,-
2,292.029724,6.244174e-102,Отклоняется,-


In [19]:
[anv.pvalue]*3

[6.244173657733121e-102, 6.244173657733121e-102, 6.244173657733121e-102]